**RICORDARSI DI ANDARE AD edit-> notebook settings-> hardware accellerator GPU**

## Clone dei repository necessari

In [0]:
! git clone https://github.com/Meyke/ProgettoSIIML.git
! git clone --recursive https://github.com/metalbubble/TRN-pytorch

Cloning into 'ProgettoSIIML'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 897 (delta 5), reused 8 (delta 4), pack-reused 883
Receiving objects: 100% (897/897), 141.53 MiB | 34.42 MiB/s, done.
Resolving deltas: 100% (220/220), done.
Cloning into 'TRN-pytorch'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 263 (delta 0), reused 6 (delta 0), pack-reused 257
Receiving objects: 100% (263/263), 69.11 KiB | 4.61 MiB/s, done.
Resolving deltas: 100% (145/145), done.
Submodule 'model_zoo' (https://github.com/yjxiong/tensorflow-model-zoo.torch) registered for path 'model_zoo'
Cloning into '/content/TRN-pytorch/model_zoo'...
remote: Enumerating objects: 621, done.        
remote: Total 621 (delta 0), reused 0 (delta 0), pack-reused 621        
Receiving objects: 100% (621/621), 43.66 MiB | 28.59 

## Import dei face embedding e dei metadati

In [0]:
% cd /content/ProgettoSIIML/deeplearning/sports_recognition

/content/ProgettoSIIML/deeplearning/sports_recognition


In [0]:
import csv
import pandas as pd
known_faces = pd.read_csv('athletes_embedding.csv', sep=',',header=None).values
athletes = pd.read_csv('athletes_metadata.csv', sep=',',header=None).values
with open('sports1.csv') as file1:
    reader1 = csv.reader(file1)
    sports1 = {rows[0]:rows[1] for rows in reader1}
with open('sports2.csv') as file2:
    reader2 = csv.reader(file2)
    sports2 = {rows[0]:rows[1] for rows in reader2}

## Download del modello

In [0]:
% cd /content/TRN-pytorch/

/content/TRN-pytorch


In [0]:
% cd pretrain
! ./download_models.sh

/content/TRN-pytorch/pretrain
--2019-04-14 15:48:32--  http://relation.csail.mit.edu/models/TRN_something_RGB_BNInception_TRNmultiscale_segment8_best.pth.tar
Resolving relation.csail.mit.edu (relation.csail.mit.edu)... 128.30.100.255
Connecting to relation.csail.mit.edu (relation.csail.mit.edu)|128.30.100.255|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52711275 (50M) [application/x-tar]
Saving to: ‘TRN_something_RGB_BNInception_TRNmultiscale_segment8_best.pth.tar’

TRN_something_RGB_B 100%[===================>]  50.27M  68.9MB/s    in 0.7s    

2019-04-14 15:48:33 (68.9 MB/s) - ‘TRN_something_RGB_BNInception_TRNmultiscale_segment8_best.pth.tar’ saved [52711275/52711275]

--2019-04-14 15:48:33--  http://relation.csail.mit.edu/models/TRN_something_RGB_BNInception_TRN_segment3_best.pth.tar
Resolving relation.csail.mit.edu (relation.csail.mit.edu)... 128.30.100.255
Connecting to relation.csail.mit.edu (relation.csail.mit.edu)|128.30.100.255|:80... connected.
HT

In [0]:
% cd /content/TRN-pytorch/model_zoo/bninception

/content/TRN-pytorch/model_zoo/bninception


In [0]:
%%writefile pytorch_load.py

import torch
from torch import nn
from .layer_factory import get_basic_layer, parse_expr
import torch.utils.model_zoo as model_zoo
import yaml


class BNInception(nn.Module):
    def __init__(self, model_path='model_zoo/bninception/bn_inception.yaml', num_classes=101,
                       weight_url='https://yjxiong.blob.core.windows.net/models/bn_inception-9f5701afb96c8044.pth'):
        super(BNInception, self).__init__()

        manifest = yaml.load(open(model_path))

        layers = manifest['layers']

        self._channel_dict = dict()

        self._op_list = list()
        for l in layers:
            out_var, op, in_var = parse_expr(l['expr'])
            if op != 'Concat':
                id, out_name, module, out_channel, in_name = get_basic_layer(l,
                                                                3 if len(self._channel_dict) == 0 else self._channel_dict[in_var[0]],
                                                                             conv_bias=True)

                self._channel_dict[out_name] = out_channel
                setattr(self, id, module)
                self._op_list.append((id, op, out_name, in_name))
            else:
                self._op_list.append((id, op, out_var[0], in_var))
                channel = sum([self._channel_dict[x] for x in in_var])
                self._channel_dict[out_var[0]] = channel

        #self.load_state_dict(torch.utils.model_zoo.load_url(weight_url))

    def forward(self, input):
        data_dict = dict()
        data_dict[self._op_list[0][-1]] = input

        def get_hook(name):

            def hook(m, grad_in, grad_out):
                print(name, grad_out[0].data.abs().mean())

            return hook
        for op in self._op_list:
            if op[1] != 'Concat' and op[1] != 'InnerProduct':
                data_dict[op[2]] = getattr(self, op[0])(data_dict[op[-1]])
                # getattr(self, op[0]).register_backward_hook(get_hook(op[0]))
            elif op[1] == 'InnerProduct':
                x = data_dict[op[-1]]
                data_dict[op[2]] = getattr(self, op[0])(x.view(x.size(0), -1))
            else:
                try:
                    data_dict[op[2]] = torch.cat(tuple(data_dict[x] for x in op[-1]), 1)
                except:
                    for x in op[-1]:
                        print(x,data_dict[x].size())
                    raise
        return data_dict[self._op_list[-1][2]]


class InceptionV3(BNInception):
    def __init__(self, model_path='model_zoo/bninception/inceptionv3.yaml', num_classes=101,
                 weight_url='https://yjxiong.blob.core.windows.net/models/inceptionv3-cuhk-0e09b300b493bc74c.pth'):
        super(InceptionV3, self).__init__(model_path=model_path, weight_url=weight_url, num_classes=num_classes)

Overwriting pytorch_load.py


## Import delle librerie

In [0]:
cd /content/TRN-pytorch

/content/TRN-pytorch


In [0]:
!pip install pafy
!pip install youtube_dl
!pip install face_recognition
import cv2
import os
import csv
import numpy as np
import pafy
import face_recognition
from PIL import Image
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import json
import re
import argparse
import functools
import subprocess
import moviepy.editor as mpy
import ast
import torchvision
import torch.nn.parallel
import torch.optim
from models import TSN
import transforms
from torch.nn import functional as F

    100% |████████████████████████████████| 1.8MB 16.6MB/s 
    100% |████████████████████████████████| 100.2MB 252kB/s 
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models
Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1892352/45929032 bytes (4.1%)5677056/45929032 bytes (12.4%)9412608/45929032 bytes (20.5%)13148160/45929032 bytes (28.6%)16932864/45929032 bytes (36.9%)20627456/45929032 bytes (44.9%)24174592/45929032 bytes (52.6%)27738112/45929032 bytes (60.4%)

## Download del video

In [0]:
url = 'https://www.youtube.com/watch?v=v--6sCcpa6s'
vPafy = pafy.new(url)
play = vPafy.getbest()
video = play.download(filepath="video.mp4")
print(vPafy.title)
print(vPafy.keywords)

Scotty James | Gold Medal | Men's Halfpipe | FIS Snowboard World Championships
['Snowboard', 'Park & Pipe']


## Riconoscimento degli atleti

In [0]:
face_locations = []
face_encodings = []
frame_number = 0
face_names = []

input_movie = cv2.VideoCapture(video)
length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))

print("Inizio processamento")
print("L'intero video contiene " + str(length) + " frame.")

while True:
    
    print("Stiamo analizzando il frame numero " + str(frame_number))
    # Grab a single frame of video
    input_movie.set(1, frame_number)
    ret, frame = input_movie.read()
    frame_number += 20

    # Quit when the input video file ends
    if not ret:
        break

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)


    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        match = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.42)
        distance = face_recognition.face_distance(known_faces, face_encoding)
        # name = None
        temp_distance = 100
        if True in match:
            for value, count in zip(match, range(len(match))):
                if value == True:
                    if distance[count] < temp_distance:
                        temp_distance = distance[count]
                        name = athletes[count]
            face_names.append(name[1])
            print("è stata riconosciuta la faccia di " + name[1])

# All done!
input_movie.release()
cv2.destroyAllWindows()
print("Fine")
print (face_names)

Inizio processamento
L'intero video contiene 8769 frame.
Stiamo analizzando il frame numero 0
Stiamo analizzando il frame numero 20
Stiamo analizzando il frame numero 40
Stiamo analizzando il frame numero 60
Stiamo analizzando il frame numero 80
Stiamo analizzando il frame numero 100
Stiamo analizzando il frame numero 120
Stiamo analizzando il frame numero 140
Stiamo analizzando il frame numero 160
Stiamo analizzando il frame numero 180
Stiamo analizzando il frame numero 200
Stiamo analizzando il frame numero 220
Stiamo analizzando il frame numero 240
Stiamo analizzando il frame numero 260
Stiamo analizzando il frame numero 280
Stiamo analizzando il frame numero 300
Stiamo analizzando il frame numero 320
Stiamo analizzando il frame numero 340
Stiamo analizzando il frame numero 360
Stiamo analizzando il frame numero 380
Stiamo analizzando il frame numero 400
Stiamo analizzando il frame numero 420
Stiamo analizzando il frame numero 440
Stiamo analizzando il frame numero 460
Stiamo analiz

KeyboardInterrupt: ignored

In [0]:
c = Counter(face_names)
output = [x[0] for x in c.items() if x[1] >= 2]
print(output)

## Riconoscimento dell'attività(sport)

In [0]:
def extract_frames(video_file, num_frames=8):
    try:
        os.makedirs(os.path.join(os.getcwd(), 'frames'))
    except OSError:
        pass

    output = subprocess.Popen(['ffmpeg', '-i', video_file],
                              stderr=subprocess.PIPE).communicate()
    # Search and parse 'Duration: 00:05:24.13,' from ffmpeg stderr.
    re_duration = re.compile('Duration: (.*?)\.')
    duration = re_duration.search(str(output[1])).groups()[0]
    temp = [int(i) for i in duration.split(":")]

    seconds = functools.reduce(lambda x, y: x * 60 + y,
                               temp)
    rate = num_frames / float(seconds)

    output = subprocess.Popen(['ffmpeg', '-i', video_file,
                               '-vf', 'fps={}'.format(rate),
                               '-vframes', str(num_frames),
                               '-loglevel', 'panic',
                               'frames/%d.jpg']).communicate()
    frame_paths = sorted([os.path.join('frames', frame)
                          for frame in os.listdir('frames')])

    frames = load_frames(frame_paths)
    subprocess.call(['rm', '-rf', 'frames'])
    return frames


def load_frames(frame_paths, num_frames=8):
    frames = [Image.open(frame).convert('RGB') for frame in frame_paths]
    if len(frames) >= num_frames:
        return frames[::int(np.ceil(len(frames) / float(num_frames)))]
    else:
        raise ValueError('Video must have at least {} frames'.format(num_frames))


def render_frames(frames, prediction):
    rendered_frames = []
    for frame in frames:
        img = np.array(frame)
        height, width, _ = img.shape
        cv2.putText(img, prediction,
                    (1, int(height / 8)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1, (255, 255, 255), 2)
        rendered_frames.append(img)
    return rendered_frames


# Get dataset categories.
categories_file = 'pretrain/moments_categories.txt'
categories = [line.rstrip() for line in open(categories_file, 'r').readlines()]
num_class = len(categories)

arch = 'InceptionV3'

# Load model.
net = TSN(num_class,
          8,
          'RGB',
          base_model= arch,
          consensus_type='TRNmultiscale',
          img_feature_dim=256, print_spec=False)

checkpoint = torch.load('pretrain/TRN_moments_RGB_InceptionV3_TRNmultiscale_segment8_best.pth.tar')
base_dict = {'.'.join(k.split('.')[1:]): v for k, v in list(checkpoint['state_dict'].items())}
net.load_state_dict(base_dict)
net.cuda().eval()

# Initialize frame transforms.
transform = torchvision.transforms.Compose([
    transforms.GroupOverSample(net.input_size, net.scale_size),
    transforms.Stack(roll=(arch in ['BNInception', 'InceptionV3'])),
    transforms.ToTorchFormatTensor(div=(arch not in ['BNInception', 'InceptionV3'])),
    transforms.GroupNormalize(net.input_mean, net.input_std),
])

# Obtain video frames
print('Extracting frames using ffmpeg...')
frames = extract_frames("video.mp4", 8)


# Make video prediction.
data = transform(frames)
input = data.view(-1, 3, data.size(1), data.size(2)).unsqueeze(0).cuda()

with torch.no_grad():
    logits = net(input)
    h_x = torch.mean(F.softmax(logits, 1), dim=0).data
    probs, idx = h_x.sort(0, True)

# Output the prediction.
video_name = "video"
predictions = []
print('RESULT ON ' + video_name)
for i in range(0, 3):
    print('{:.3f} -> {}'.format(probs[i], categories[idx[i]]))
    predictions.append(categories[idx[i]])
    
def get_result(results):
  for result in results:
    if sports1.get(result) != None:
      return sports1.get(result)
  for result in results:
    if sports2.get(result) != None:
      return sports2.get(result)
if get_result(predictions) != None:
  print("Sport recognized is " + ast.literal_eval(get_result(predictions))[0])
  output = output + ast.literal_eval(get_result(predictions))
else:
  print("No sport recognized")

  normal(self.new_fc.weight, 0, std)

  constant(self.new_fc.bias, 0)



Multi-Scale Temporal Relation Network Module in use ['8-frame relation', '7-frame relation', '6-frame relation', '5-frame relation', '4-frame relation', '3-frame relation', '2-frame relation']


  "please use transforms.Resize instead.")



Freezing BatchNorm2D except the first one.
Extracting frames using ffmpeg...
RESULT ON video
0.732 -> skiing
0.035 -> skating
0.029 -> racing
No sport recognized


## Input per PredictionIO

In [0]:
output = [x.lower() for x in output]
print("Input for recommender system is: " + str(output))

In [0]:
!nvidia-smi

Mon Apr 15 16:23:14 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    71W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
! conda list cudnn

In [0]:
from tensorflow.python.platform import build_info as tf_build_info
print(tf_build_info.cuda_version_number)
# 9.0 in v1.10.0
print(tf_build_info.cudnn_version_number)